In [67]:
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime
from tensorflow import keras
import os
import re
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

In [68]:
# Load all files from a directory in a DataFrame.
def load_directory_data(directory):
    data = {}
    data["sentence"] = []
    data["sentiment"] = []
    for file_path in os.listdir(directory):
        with tf.gfile.GFile(os.path.join(directory, file_path), "r") as f:
            data["sentence"].append(f.read())
            data["sentiment"].append(
                re.match("\d+_(\d+)\.txt", file_path).group(1))
    return pd.DataFrame.from_dict(data)


# Merge positive and negative examples, add a polarity column and shuffle.
def load_dataset(directory):
    pos_df = load_directory_data(os.path.join(directory, "pos"))
    neg_df = load_directory_data(os.path.join(directory, "neg"))
    pos_df["polarity"] = 1
    neg_df["polarity"] = 0
    return pd.concat([pos_df, neg_df]).sample(frac=1).reset_index(drop=True)


# Download and process the dataset files.
def download_and_load_datasets(force_download=False):
    dataset = tf.keras.utils.get_file(
        fname="aclImdb.tar.gz",
        origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz",
        extract=True)
    print(dataset)
    print(os.path.dirname(dataset))

    train_df = load_dataset(
        os.path.join(os.path.dirname(dataset), "aclImdb", "train"))
    test_df = load_dataset(
        os.path.join(os.path.dirname(dataset), "aclImdb", "test"))

    return train_df, test_df

In [69]:
# Reduce logging output.
train, test = download_and_load_datasets()

C:\Users\wangfabei\.keras\datasets\aclImdb.tar.gz
C:\Users\wangfabei\.keras\datasets


In [70]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
sentence     25000 non-null object
sentiment    25000 non-null object
polarity     25000 non-null int64
dtypes: int64(1), object(2)
memory usage: 586.0+ KB


In [71]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
sentence     25000 non-null object
sentiment    25000 non-null object
polarity     25000 non-null int64
dtypes: int64(1), object(2)
memory usage: 586.0+ KB


In [72]:
test.describe()

,polarity
count,25000.00000
mean,0.50000
std,0.50001
min,0.00000
25%,0.00000
50%,0.50000
75%,1.00000
max,1.00000


In [73]:
train = train.sample(5000)
test = test.sample(5000)

In [74]:
train.head(10)

,sentence,sentiment,polarity
9392,This is one of the worst movies I've seen in a...,1,0
15543,I gave this a four purely out of its historica...,4,0
3720,I have no idea why this flick is getting such ...,8,1
17458,I just saw this film yesterday.<br /><br />My ...,1,0
4273,I have seen the movie at the Viennale a few ye...,9,1
6291,I bought this from Blockbuster for 99p. The gu...,1,0
24977,I remember watching this is its original airin...,9,1
14764,This picture reminds me of a Keneth More pictu...,3,0
22079,"I wonder who, how and more importantly why the...",3,0
3869,"As a Pagan, I must say this movie has little i...",9,1


In [75]:
train.columns

Index(['sentence', 'sentiment', 'polarity'], dtype='object')

In [76]:
DATA_COLUMN = 'sentence'
LABEL_COLUMN = 'polarity'
# label_list is the list of labels, i.e. True, False or 0, 1 or 'dog', 'cat'
label_list = [0, 1]

In [77]:
train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

In [78]:
type(train_InputExamples)

pandas.core.series.Series

In [79]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
    """Get the vocab file and casing info from the Hub module."""
    with tf.Graph().as_default():
        bert_module = hub.Module(BERT_MODEL_HUB)
        tokenization_info = bert_module(signature="tokenization_info",
                                        as_dict=True)
        print(tokenization_info)
        with tf.Session() as sess:
            vocab_file, do_lower_case = sess.run([
                tokenization_info["vocab_file"],
                tokenization_info["do_lower_case"]
            ])

    return bert.tokenization.FullTokenizer(vocab_file=vocab_file,
                                           do_lower_case=do_lower_case),vocab_file,do_lower_case

In [80]:
tokenizer,vocab_file ,do_lower_case= create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0417 10:44:33.744045 15268 saver.py:1483] Saver not created because there are no variables in the graph to restore


{'do_lower_case': <tf.Tensor 'module_apply_tokenization_info/Const:0' shape=() dtype=bool>, 'vocab_file': <tf.Tensor 'module_apply_tokenization_info/vocab_file:0' shape=() dtype=string>}


In [81]:
tokenizer.tokenize("This here's an example of using the BERT tokenizer")

['this',
 'here',
 "'",
 's',
 'an',
 'example',
 'of',
 'using',
 'the',
 'bert',
 'token',
 '##izer']

In [82]:
# vocab_file
do_lower_case

True

In [83]:
tokenizer.vocab
len(tokenizer.inv_vocab)

30522

In [84]:
MAX_SEQ_LENGTH = 128
# Convert our train and test features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 5000


I0417 10:44:34.601163 15268 run_classifier.py:774] Writing example 0 of 5000


INFO:tensorflow:*** Example ***


I0417 10:44:34.604164 15268 run_classifier.py:461] *** Example ***


INFO:tensorflow:guid: None


...

I0417 10:44:56.233309 15268 run_classifier.py:466] input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


I0417 10:44:56.234304 15268 run_classifier.py:467] segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 0 (id = 0)


I0417 10:44:56.237305 15268 run_classifier.py:468] label: 0 (id = 0)


In [85]:

print(train_features[0].input_ids)
print('--------')
print(train_features[0].input_mask)
print('--------')
print(train_features[0].segment_ids)
print('--------')
print(train_features[0].label_id)

[101, 2023, 2003, 2028, 1997, 1996, 5409, 5691, 1045, 1005, 2310, 2464, 1999, 1037, 2146, 2051, 1012, 1996, 2466, 2001, 11771, 1010, 1996, 7982, 2001, 2012, 3217, 18436, 1998, 1996, 3772, 10654, 8029, 1012, 1045, 1005, 1049, 2025, 2469, 2065, 2023, 3185, 2001, 1996, 2765, 1997, 1037, 2143, 2082, 19453, 2622, 1010, 2021, 2009, 5121, 2209, 2066, 2028, 1012, 2009, 2003, 2025, 2130, 3391, 3144, 1999, 2049, 2430, 9530, 3401, 4183, 1997, 2667, 2000, 3711, 2004, 1037, 2309, 7142, 2202, 1012, 1996, 2040, 24303, 3070, 9876, 4950, 6090, 2015, 2024, 1037, 10036, 1998, 27776, 3089, 24965, 2126, 1997, 6318, 7659, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
--------
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [86]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
    """Creates a classification model."""

    bert_module = hub.Module(BERT_MODEL_HUB, trainable=True)
    bert_inputs = dict(input_ids=input_ids,
                       input_mask=input_mask,
                       segment_ids=segment_ids)
    bert_outputs = bert_module(inputs=bert_inputs,
                               signature="tokens",
                               as_dict=True)
    print("bert_outputs")
    print(bert_outputs)

    # Use "pooled_output" for classification tasks on an entire sentence.
    # Use "sequence_outputs" for token-level output.
    output_layer = bert_outputs["pooled_output"]

    hidden_size = output_layer.shape[-1].value

    # Create our own layer to tune for politeness data.
    output_weights = tf.get_variable(
        "output_weights", [num_labels, hidden_size],
        initializer=tf.truncated_normal_initializer(stddev=0.02))

    output_bias = tf.get_variable("output_bias", [num_labels],
                                  initializer=tf.zeros_initializer())

    with tf.variable_scope("loss"):

        # Dropout helps prevent overfitting
        output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

        logits = tf.matmul(output_layer, output_weights, transpose_b=True)
        logits = tf.nn.bias_add(logits, output_bias)
        log_probs = tf.nn.log_softmax(logits, axis=-1)

        # Convert labels into one-hot encoding
        one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

        predicted_labels = tf.squeeze(
            tf.argmax(log_probs, axis=-1, output_type=tf.int32))
        # If we're predicting, we want predicted labels and the probabiltiies.
        if is_predicting:
            return (predicted_labels, log_probs)

        # If we're train/eval, compute loss between predicted and actual label
        per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
        loss = tf.reduce_mean(per_example_loss)
        return (loss, predicted_labels, log_probs)

In [87]:
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
    """Returns `model_fn` closure for TPUEstimator."""

    def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
        """The `model_fn` for TPUEstimator."""

        input_ids = features["input_ids"]
        input_mask = features["input_mask"]
        segment_ids = features["segment_ids"]
        label_ids = features["label_ids"]

        is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)

        # TRAIN and EVAL
        if not is_predicting:

            (loss, predicted_labels,
             log_probs) = create_model(is_predicting, input_ids, input_mask,
                                       segment_ids, label_ids, num_labels)

            train_op = bert.optimization.create_optimizer(loss,
                                                          learning_rate,
                                                          num_train_steps,
                                                          num_warmup_steps,
                                                          use_tpu=False)

            # Calculate evaluation metrics.
            def metric_fn(label_ids, predicted_labels):
                accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
                f1_score = tf.contrib.metrics.f1_score(label_ids,
                                                       predicted_labels)
                auc = tf.metrics.auc(label_ids, predicted_labels)
                recall = tf.metrics.recall(label_ids, predicted_labels)
                precision = tf.metrics.precision(label_ids, predicted_labels)
                true_pos = tf.metrics.true_positives(label_ids,
                                                     predicted_labels)
                true_neg = tf.metrics.true_negatives(label_ids,
                                                     predicted_labels)
                false_pos = tf.metrics.false_positives(label_ids,
                                                       predicted_labels)
                false_neg = tf.metrics.false_negatives(label_ids,
                                                       predicted_labels)
                return {
                    "eval_accuracy": accuracy,
                    "f1_score": f1_score,
                    "auc": auc,
                    "precision": precision,
                    "recall": recall,
                    "true_positives": true_pos,
                    "true_negatives": true_neg,
                    "false_positives": false_pos,
                    "false_negatives": false_neg
                }

            eval_metrics = metric_fn(label_ids, predicted_labels)

            if mode == tf.estimator.ModeKeys.TRAIN:
                return tf.estimator.EstimatorSpec(mode=mode,
                                                  loss=loss,
                                                  train_op=train_op)
            else:
                return tf.estimator.EstimatorSpec(mode=mode,
                                                  loss=loss,
                                                  eval_metric_ops=eval_metrics)
        else:
            (predicted_labels,
             log_probs) = create_model(is_predicting, input_ids, input_mask,
                                       segment_ids, label_ids, num_labels)

            predictions = {
                'probabilities': log_probs,
                'labels': predicted_labels
            }
            return tf.estimator.EstimatorSpec(mode, predictions=predictions)

    # Return the actual model function in the closure
    return model_fn

In [88]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

In [89]:
# Compute # train and warmup steps from batch size
import os
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)


In [90]:
curr = r'F:\testDemo\AI\NER\ChineseNER-based-BERT'

In [91]:
OUTPUT_DIR=curr

In [92]:
# Specify outpit directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [93]:
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})


INFO:tensorflow:Using config: {'_model_dir': 'F:\\testDemo\\AI\\NER\\ChineseNER-based-BERT', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000025007F6B8D0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


I0417 10:45:17.957018 15268 estimator.py:201] Using config: {'_model_dir': 'F:\\testDemo\\AI\\NER\\ChineseNER-based-BERT', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000025007F6B8D0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [47]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

In [48]:
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
INFO:tensorflow:Calling model_fn.


I0415 19:34:15.338690 15268 estimator.py:1111] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0415 19:34:18.137732 15268 saver.py:1483] Saver not created because there are no variables in the graph to restore


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


...

INFO:tensorflow:Saving checkpoints for 468 into F:\testDemo\AI\NER\ChineseNER-based-BERT\model.ckpt.


I0415 22:37:51.994027 15268 basic_session_run_hooks.py:594] Saving checkpoints for 468 into F:\testDemo\AI\NER\ChineseNER-based-BERT\model.ckpt.


INFO:tensorflow:Loss for final step: 0.12777634.


I0415 22:38:06.528198 15268 estimator.py:359] Loss for final step: 0.12777634.


Training took time  3:03:53.472833


In [49]:
test_input_fn = run_classifier.input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [50]:
estimator.evaluate(input_fn=test_input_fn, steps=None)

INFO:tensorflow:Calling model_fn.


I0415 22:38:09.039033 15268 estimator.py:1111] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0415 22:38:12.089213 15268 saver.py:1483] Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving dict for global step 468: auc = 0.87043816, eval_accuracy = 0.8704, f1_score = 0.87183535, false_negatives = 289.0, false_positives = 359.0, global_step = 468, loss = 0.5175862, precision = 0.85992974, recall = 0.8840754, true_negatives = 2148.0, true_positives = 2204.0


I0415 22:59:12.372771 15268 estimator.py:1979] Saving dict for global step 468: auc = 0.87043816, eval_accuracy = 0.8704, f1_score = 0.87183535, false_negatives = 289.0, false_positives = 359.0, global_step = 468, loss = 0.5175862, precision = 0.85992974, recall = 0.8840754, true_negatives = 2148.0, true_positives = 2204.0


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 468: F:\testDemo\AI\NER\ChineseNER-based-BERT\model.ckpt-468


I0415 22:59:15.285777 15268 estimator.py:2039] Saving 'checkpoint_path' summary for global step 468: F:\testDemo\AI\NER\ChineseNER-based-BERT\model.ckpt-468


{'auc': 0.87043816,
 'eval_accuracy': 0.8704,
 'f1_score': 0.87183535,
 'false_negatives': 289.0,
 'false_positives': 359.0,
 'loss': 0.5175862,
 'precision': 0.85992974,
 'recall': 0.8840754,
 'true_negatives': 2148.0,
 'true_positives': 2204.0,
 'global_step': 468}

In [53]:
def getPrediction(in_sentences):
    labels = ["Negative", "Positive"]
    input_examples = [
        run_classifier.InputExample(guid="", text_a=x, text_b=None, label=0)
        for x in in_sentences
    ]  # here, "" is just a dummy label
    input_features = run_classifier.convert_examples_to_features(
        input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
    predict_input_fn = run_classifier.input_fn_builder(
        features=input_features,
        seq_length=MAX_SEQ_LENGTH,
        is_training=False,
        drop_remainder=False)
    predictions = estimator.predict(predict_input_fn)
    return [(sentence, prediction['probabilities'],
             labels[prediction['labels']])
            for sentence, prediction in zip(in_sentences, predictions)]


In [54]:
pred_sentences = [
  "That movie was absolutely awful",
  "The acting was a bit lacking",
  "The film was creative and surprising",
  "Absolutely fantastic!"
]

In [55]:
predictions = getPrediction(pred_sentences)

INFO:tensorflow:Writing example 0 of 4


I0416 16:02:42.124437 15268 run_classifier.py:774] Writing example 0 of 4


INFO:tensorflow:*** Example ***


I0416 16:02:42.125407 15268 run_classifier.py:461] *** Example ***


INFO:tensorflow:guid: 


...

I0416 16:02:46.371434 15268 saver.py:1270] Restoring parameters from F:\testDemo\AI\NER\ChineseNER-based-BERT\model.ckpt-468


INFO:tensorflow:Running local_init_op.


I0416 16:02:47.429471 15268 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0416 16:02:47.529442 15268 session_manager.py:493] Done running local_init_op.


In [56]:
predictions

[('That movie was absolutely awful',
  array([-1.8512745e-03, -6.2928047e+00], dtype=float32),
  'Negative'),
 ('The acting was a bit lacking',
  array([-2.1979478e-03, -6.1213036e+00], dtype=float32),
  'Negative'),
 ('The film was creative and surprising',
  array([-5.5807433e+00, -3.7769189e-03], dtype=float32),
  'Positive'),
 ('Absolutely fantastic!',
  array([-5.6152534e+00, -3.6485312e-03], dtype=float32),
  'Positive')]